In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings
import os
warnings.filterwarnings('ignore')

загрузка графа, неважно

In [2]:
# ЗАГРУЗКА ГРАФА ИЗ PICKLE

def load_graph_pickle(pickle_file):
    """Загружает граф из pickle файла"""
    print(f"📥 Загрузка графа из {pickle_file}...")
    with open(pickle_file, 'rb') as f:
        G = pickle.load(f)
    print(f"   ✓ Загружено: {G.number_of_nodes()} узлов, {G.number_of_edges()} рёбер")
    return G

тут считаем кратчайший путь для каждой пары из тех 5000 что мы выбирали "центр"-"периферия" (называем их O-D = origin-destination), в качестве весов ребер участвует как раз время (travel_time). Для пар, которые не связаны между собой, ставим время равное бесконечности

In [3]:
# 1. ПРЕДВАРИТЕЛЬНЫЙ РАСЧЕТ: Исходные кратчайшие пути d_ij

def calculate_initial_distances(graph, od_pairs_file, output_file='initial_distances.csv'):
    """
    Рассчитывает исходное кратчайшее время в пути для всех O-D пар.
    """
    print("\n" + "="*80)
    print("📊 ШАГ 1: РАСЧЕТ ИСХОДНЫХ РАССТОЯНИЙ")
    print("="*80)

    print(f"🚀 Загрузка O-D пар из {od_pairs_file}...")
    od_pairs = pd.read_csv(od_pairs_file)

    results = []
    print(f"📐 Расчет кратчайших путей для {len(od_pairs)} пар...")

    for idx, row in tqdm(od_pairs.iterrows(), total=len(od_pairs), desc="Обработка пар"):
        origin = row['node_id_origin']
        dest = row['node_id_dest']

        try:
            # Кратчайший путь по весу travel_time
            distance = nx.shortest_path_length(graph, origin, dest, weight='travel_time')
            results.append({
                'origin': origin,
                'dest': dest,
                'distance_original': distance
            })
        except nx.NetworkXNoPath:
            # Если пути нет, расстояние = бесконечность
            results.append({
                'origin': origin,
                'dest': dest,
                'distance_original': np.inf
            })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)

    # Статистика
    valid_distances = df[df['distance_original'] != np.inf]['distance_original']
    print(f"\n✅ Результаты сохранены в {output_file}")
    print(f"📈 Статистика:")
    print(f"   - Всего пар: {len(df)}")
    print(f"   - Связных пар: {len(valid_distances)}")
    print(f"   - Несвязных пар: {len(df) - len(valid_distances)}")
    print(f"   - Среднее время: {valid_distances.mean():.2f} мин")
    print(f"   - Медиана: {valid_distances.median():.2f} мин")
    print(f"   - Макс: {valid_distances.max():.2f} мин")

    return df

критичность ребра W_e (может назвать по другому, смотря использовалось ли где то такое). 

Формула: $W_e=\sum_{(i,j)\in OD} max\left(0, \frac{d_{ij}^{new}(e) - d_{ij}^{original}}{d_{ij}^{original}} \right) \cdot (\text{Спрос}_{ij})$, где 

- OD - пара центр-периферия
- $d_{ij}^{original}$ - испходное кратчайшее время в пути между i и j
- $d_{ij}^{new}(e)$ - кратчайшее время в пути между i и j после удаления ребра e
- Название метрики - Stretch factor - показывает, насколько процентов увеличился путь после удаления ребра. Таким образом если путь сильно увеличился, значит ребро важное
- Спрос - коэффициент (пока не использовался, но может сделаю что-то), но лучше пока его в формуле не писать вообще

In [4]:
# 2. РАСЧЕТ КРИТИЧНОСТИ РЕБРА (W_e) - STRETCH FACTOR
def calculate_edge_criticality(graph, od_pairs_df, demand_dict=None,
                                    output_file="criticality_scores.csv",
                                    checkpoint_interval=500):
    """
    Быстрый расчёт критичности ребра через предвычисленные SPT деревья.
    """

    print("\n" + "=" * 80)
    print("⚡ ШАГ 2: РАСЧЁТ КРИТИЧНОСТИ РЁБЕР (W_e)")
    print("=" * 80)

    edges = list(graph.edges(keys=True))
    criticality_scores = []

    # Если спрос не задан, все пары имеют одинаковый спрос = 1
    if demand_dict is None:
        demand_dict = {(row['origin'], row['dest']): 1
                       for _, row in od_pairs_df.iterrows()}

    # Фильтруем только связные пары
    valid_od_pairs = od_pairs_df[od_pairs_df['distance_original'] != np.inf]
    print(f"📦 Всего ребер для анализа: {len(edges)}")
    print(f"📍 Валидных O-D пар: {len(valid_od_pairs)}/{len(od_pairs_df)}")

    # Предвычисляем SPT для ускорения
    sources = valid_od_pairs['origin'].unique()
    print(f"🧠 Предвычисление SPT от {len(sources)} источников...")

    # Словарь: dist[source][node] = расстояние, parent[source][node] = родитель
    dist = {}
    parent = {}

    for s in tqdm(sources, desc="SPT деревья"):
        lengths, paths = nx.single_source_dijkstra(graph, s, weight='travel_time')
        dist[s] = lengths

        # Строим parent-таблицу для быстрой проверки
        p = {}
        for t, path in paths.items():
            if len(path) >= 2:
                p[t] = path[-2]  # предыдущий узел = родитель
        parent[s] = p

    print("✅ SPT построены")
    print()

    # Функция для проверки, использует ли путь s→t ребро (u,v)
    def uses_edge(s, t, u, v):
        """Проверяет, идет ли кратчайший путь s→t через ребро u→v"""
        if s not in parent or t not in parent[s]:
            return False

        curr = t
        while curr != s and curr in parent[s]:
            p = parent[s][curr]
            # Проверяем ребро в обе стороны (для MultiDiGraph)
            if (p == u and curr == v) or (p == v and curr == u):
                return True
            curr = p
        return False

    print(f"🔍 Расчёт критичности {len(edges)} рёбер...")
    print()

    for edge_idx, (u, v, key) in enumerate(tqdm(edges, desc="Анализ ребер")):
        total_stretch = 0
        affected_pairs = 0

        # Проходим только по парам, которые могут использовать это ребро
        for _, row in valid_od_pairs.iterrows():
            s = row['origin']
            t = row['dest']
            d_original = row['distance_original']

            # Быстрая проверка: использует ли кратчайший путь это ребро?
            if not uses_edge(s, t, u, v):
                continue  # Ребро не влияет на эту пару - пропускаем

            affected_pairs += 1

            # Ребро влияет - нужно пересчитать путь без него
            try:
                # ПРАВИЛЬНО: создаем копию графа
                G_temp = graph.copy()
                G_temp.remove_edge(u, v, key)

                d_new = nx.shortest_path_length(G_temp, s, t, weight='travel_time')
                stretch = max(0, (d_new - d_original) / d_original)

            except nx.NetworkXNoPath:
                # Путь разорван - большой штраф
                stretch = 10.0

            demand = demand_dict.get((s, t), 1)
            total_stretch += stretch * demand

        criticality_scores.append({
            'edge_u': u,
            'edge_v': v,
            'edge_key': key,
            'W_e': total_stretch,
            'affected_pairs': affected_pairs  # Для отладки
        })

        # Промежуточное сохранение
        if (edge_idx + 1) % checkpoint_interval == 0:
            temp_df = pd.DataFrame(criticality_scores)
            temp_file = f'checkpoint_{output_file}'
            temp_df.to_csv(temp_file, index=False)
            print(f"\n💾 Checkpoint: обработано {edge_idx + 1}/{len(edges)} ребер")

    # Сортируем по убыванию критичности
    df = pd.DataFrame(criticality_scores)
    df = df.sort_values('W_e', ascending=False).reset_index(drop=True)
    df.to_csv(output_file, index=False)

    print(f"\n✅ Критичность ребер сохранена в {output_file}")
    print(f"\n🔝 Топ-10 самых критичных ребер:")
    print(df.head(10)[['edge_u', 'edge_v', 'W_e', 'affected_pairs']].to_string())
    print(f"\n📊 Статистика W_e:")
    print(f"   - Среднее: {df['W_e'].mean():.4f}")
    print(f"   - Медиана: {df['W_e'].median():.4f}")
    print(f"   - Макс: {df['W_e'].max():.4f}")
    print(f"   - Мин: {df['W_e'].min():.4f}")
    print(f"   - Ребер с W_e > 0: {(df['W_e'] > 0).sum()}/{len(df)}")

    return df


Метрики связности (доступности разных частей) графа - LCC и Efficiency. LCC - доля узлов которые в наибольшей компоненте. 

$Efficiency = \frac{1}{n(n-1)} \sum _{i \neq j} \frac{1}{d_{ij}}$

- n - общее колво узлов
- $d_{ij}$ - кратчайшее время в пути между i и j. Если пути нет, то 1/d=0

LCC проверяет формальную связность, а Efficiency как раз фактическую по идее, объяснить как то надо я хз. Чисто LCC не используем за счет того что это город, и полный разрыв не очень логично проверять. Соответственно в результате графики разные будут

In [5]:

# 3. МЕТРИКИ: LCC и GLOBAL EFFICIENCY

def calculate_lcc(graph):
    """
    Рассчитывает размер крупнейшей связной компоненты (LCC).
    Возвращает долю узлов в LCC от общего числа узлов.
    """
    if graph.number_of_nodes() == 0:
        return 0.0

    # Для направленного графа используем weakly connected
    if graph.is_directed():
        largest_cc = max(nx.weakly_connected_components(graph), key=len)
    else:
        largest_cc = max(nx.connected_components(graph), key=len)

    return len(largest_cc) / graph.number_of_nodes()


def calculate_global_efficiency(graph, sample_size=None):
    """
    Рассчитывает глобальную эффективность (E).

    E = 1 / (N*(N-1)) * Σ (1 / d_ij)

    Args:
        graph: NetworkX граф
        sample_size: если задано, использует выборку узлов для ускорения
    """
    nodes = list(graph.nodes())
    N = len(nodes)

    if N <= 1:
        return 0.0

    # Для больших графов используем выборку
    if sample_size and N > sample_size:
        nodes = np.random.choice(nodes, sample_size, replace=False)
        N = len(nodes)

    total_inv_distance = 0
    count = 0

    for i in range(N):
        # Кратчайшие пути от узла i ко всем остальным
        try:
            lengths = nx.single_source_dijkstra_path_length(
                graph, nodes[i], weight='travel_time'
            )

            for j in range(i + 1, N):
                if nodes[j] in lengths:
                    d_ij = lengths[nodes[j]]
                    if d_ij > 0:
                        total_inv_distance += 1.0 / d_ij
                        count += 1
        except:
            continue

    if count == 0:
        return 0.0

    efficiency = total_inv_distance / (N * (N - 1))
    return efficiency


Далее Target атака - отключаем поочередно узлы по убыванию их важности (метрика W_e)

In [6]:
# 4. СИМУЛЯЦИЯ: ЦЕЛЕВАЯ АТАКА (Targeted Attack)

def simulate_targeted_attack(graph, criticality_df, steps=20, efficiency_sample=500):
    """
    Симулирует целевую атаку: удаляет ребра по убыванию критичности.

    Args:
        graph: исходный граф
        criticality_df: DataFrame с колонками edge_u, edge_v, edge_key, W_e
        steps: количество шагов (процентов удаления)
        efficiency_sample: размер выборки для расчета Efficiency
    """
    print("\n" + "="*80)
    print("🎯 ШАГ 3: СИМУЛЯЦИЯ ЦЕЛЕВОЙ АТАКИ")
    print("="*80)

    G = graph.copy()
    total_edges = G.number_of_edges()
    edges_to_remove = criticality_df[['edge_u', 'edge_v', 'edge_key']].values

    results = []

    # Исходное состояние
    print("📊 Расчет исходных метрик...")
    lcc_0 = calculate_lcc(G)
    eff_0 = calculate_global_efficiency(G, sample_size=efficiency_sample)
    results.append({
        'fraction_removed': 0.0,
        'strategy': 'Targeted',
        'metric_type': 'LCC',
        'value': lcc_0
    })
    results.append({
        'fraction_removed': 0.0,
        'strategy': 'Targeted',
        'metric_type': 'Efficiency',
        'value': eff_0
    })

    print(f"   Исходная LCC: {lcc_0:.4f}")
    print(f"   Исходная Efficiency: {eff_0:.6f}")
    print(f"\n🔨 Начинаю удаление ребер по критичности...")
    print(f"   Всего шагов: {steps}")
    print(f"   Ребер на шаг: ~{total_edges // steps}")

    # Удаляем ребра шагами
    step_size = len(edges_to_remove) // steps

    for step in tqdm(range(1, steps + 1), desc="Целевая атака"):
        # Удаляем следующую порцию ребер
        start_idx = (step - 1) * step_size
        end_idx = min(step * step_size, len(edges_to_remove))

        for i in range(start_idx, end_idx):
            u, v, key = edges_to_remove[i]
            if G.has_edge(u, v, key):
                G.remove_edge(u, v, key)

        fraction = step / steps

        # Измеряем метрики
        lcc = calculate_lcc(G)
        eff = calculate_global_efficiency(G, sample_size=efficiency_sample)

        results.append({
            'fraction_removed': fraction,
            'strategy': 'Targeted',
            'metric_type': 'LCC',
            'value': lcc
        })
        results.append({
            'fraction_removed': fraction,
            'strategy': 'Targeted',
            'metric_type': 'Efficiency',
            'value': eff
        })

        if step % 5 == 0 or step == steps:
            print(f"   Шаг {step}/{steps} ({fraction*100:.0f}%): LCC={lcc:.4f}, Eff={eff:.6f}")

    print(f"\n✅ Целевая атака завершена!")
    return pd.DataFrame(results)

Далее случайный отказ - рандомно отключаем узлы поочереди, и усредняем значения каждой из метрик LCC и Efficiency

In [7]:
# 5. СИМУЛЯЦИЯ: СЛУЧАЙНЫЙ ОТКАЗ (Random Failure)

def simulate_random_failure(graph, steps=20, runs=50, efficiency_sample=500):
    """
    Симулирует случайный отказ: удаляет случайные ребра.
    Повторяет runs раз и усредняет результаты.

    Args:
        graph: исходный граф
        steps: количество шагов (процентов удаления)
        runs: количество повторений для усреднения
        efficiency_sample: размер выборки для расчета Efficiency
    """
    print("\n" + "="*80)
    print("🎲 ШАГ 4: СИМУЛЯЦИЯ СЛУЧАЙНОГО ОТКАЗА")
    print("="*80)
    print(f"   Количество прогонов: {runs}")
    print(f"   Шагов на прогон: {steps}")

    all_results = []

    for run in tqdm(range(runs), desc="Прогоны"):
        G = graph.copy()
        total_edges = G.number_of_edges()
        all_edges = list(G.edges(keys=True))

        # Перемешиваем ребра случайным образом
        np.random.shuffle(all_edges)

        run_results = []

        # Исходное состояние
        lcc_0 = calculate_lcc(G)
        eff_0 = calculate_global_efficiency(G, sample_size=efficiency_sample)
        run_results.append({
            'fraction_removed': 0.0,
            'run': run,
            'LCC': lcc_0,
            'Efficiency': eff_0
        })

        # Удаляем ребра шагами
        step_size = total_edges // steps

        for step in range(1, steps + 1):
            start_idx = (step - 1) * step_size
            end_idx = min(step * step_size, len(all_edges))

            for i in range(start_idx, end_idx):
                u, v, key = all_edges[i]
                if G.has_edge(u, v, key):
                    G.remove_edge(u, v, key)

            fraction = step / steps
            lcc = calculate_lcc(G)
            eff = calculate_global_efficiency(G, sample_size=efficiency_sample)

            run_results.append({
                'fraction_removed': fraction,
                'run': run,
                'LCC': lcc,
                'Efficiency': eff
            })

        all_results.extend(run_results)

    # Усредняем результаты по всем прогонам
    df_all = pd.DataFrame(all_results)
    df_avg = df_all.groupby('fraction_removed').agg({
        'LCC': 'mean',
        'Efficiency': 'mean'
    }).reset_index()

    # Преобразуем в нужный формат
    results = []
    for _, row in df_avg.iterrows():
        results.append({
            'fraction_removed': row['fraction_removed'],
            'strategy': 'Random',
            'metric_type': 'LCC',
            'value': row['LCC']
        })
        results.append({
            'fraction_removed': row['fraction_removed'],
            'strategy': 'Random',
            'metric_type': 'Efficiency',
            'value': row['Efficiency']
        })

    print(f"\n✅ Случайный отказ завершен! Усреднено {runs} прогонов")
    return pd.DataFrame(results)

Дальше просто запуск этих функций

# ДЛЯ СТАТЬИ ДАЛЬШЕ НЕ НАДО ИДТИ

In [8]:
# 6. ГЛАВНАЯ ФУНКЦИЯ: ПОЛНЫЙ ПАЙПЛАЙН

def main_pipeline(graph_name='zmr', data_dir='map_data', output_dir='results'):
    """
    Полный пайплайн анализа перколяции.

    Args:
        graph_name: имя графа ('moscow', 'sao', 'zmr')
        data_dir: директория с данными от Олега
        output_dir: директория для сохранения результатов
    """
    os.makedirs(output_dir, exist_ok=True)

    print("="*80)
    print(f"🚀 СТАРТ АНАЛИЗА ФУНКЦИОНАЛЬНОЙ УСТОЙЧИВОСТИ: {graph_name.upper()}")
    print("="*80)

    # Пути к файлам
    graph_file = os.path.join(data_dir, f'{graph_name}.pkl')
    od_pairs_file = os.path.join(data_dir, f'{graph_name}_od_pairs.csv')

    # Загрузка графа
    G = load_graph_pickle(graph_file)

    # Создаем поддиректорию для результатов этого графа
    graph_output_dir = os.path.join(output_dir, graph_name)
    os.makedirs(graph_output_dir, exist_ok=True)

    # 1. Исходные расстояния
    initial_dist_file = os.path.join(graph_output_dir, 'initial_distances.csv')
    if not os.path.exists(initial_dist_file):
        od_df = calculate_initial_distances(G, od_pairs_file, initial_dist_file)
    else:
        print(f"\n✓ Найден существующий {initial_dist_file}, загружаю...")
        od_df = pd.read_csv(initial_dist_file)
        print(f"   Загружено {len(od_df)} O-D пар")

    # 2. Критичность ребер
    criticality_file = os.path.join(graph_output_dir, 'criticality_scores.csv')
    if not os.path.exists(criticality_file):
        crit_df = calculate_edge_criticality(G, od_df, output_file=criticality_file)



    else:
        print(f"\n✓ Найден существующий {criticality_file}, загружаю...")
        crit_df = pd.read_csv(criticality_file)
        print(f"   Загружено критичностей для {len(crit_df)} ребер")

    # 3. Целевая атака
    results_targeted = simulate_targeted_attack(G, crit_df, steps=20, efficiency_sample=500)

    # 4. Случайный отказ
    results_random = simulate_random_failure(G, steps=20, runs=50, efficiency_sample=500)

    # 5. Объединение результатов
    print("\n" + "="*80)
    print("📊 ФИНАЛИЗАЦИЯ РЕЗУЛЬТАТОВ")
    print("="*80)
    final_results = pd.concat([results_targeted, results_random], ignore_index=True)

    output_file = os.path.join(graph_output_dir, 'percolation_results.csv')
    final_results.to_csv(output_file, index=False)

    print(f"✅ Результаты сохранены в {output_file}")

    # Сохраняем также краткую статистику
    stats_file = os.path.join(graph_output_dir, 'summary.txt')
    with open(stats_file, 'w', encoding='utf-8') as f:
        f.write(f"Граф: {graph_name}\n")
        f.write(f"Узлов: {G.number_of_nodes()}\n")
        f.write(f"Ребер: {G.number_of_edges()}\n")
        f.write(f"O-D пар: {len(od_df)}\n")
        f.write(f"\nИсходные метрики:\n")
        f.write(f"LCC: {results_targeted[results_targeted['fraction_removed']==0]['value'].iloc[0]:.4f}\n")
        f.write(f"Efficiency: {results_targeted[results_targeted['fraction_removed']==0]['value'].iloc[1]:.6f}\n")

    print(f"📄 Краткая статистика сохранена в {stats_file}")

    # Строим график
    plot_results(final_results, graph_output_dir, graph_name)

    print("\n" + "="*80)
    print(f"✅ АНАЛИЗ ЗАВЕРШЕН ДЛЯ {graph_name.upper()}!")
    print("="*80)

    return final_results


# ВИЗУАЛИЗАЦИЯ

def plot_results(results, output_dir, graph_name):
    """Строит графики перколяции"""
    import matplotlib.pyplot as plt

    print("\n📈 Построение графиков...")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # LCC
    for strategy in ['Targeted', 'Random']:
        data = results[(results['strategy'] == strategy) &
                      (results['metric_type'] == 'LCC')]
        ax1.plot(data['fraction_removed'], data['value'],
                marker='o', label=strategy, linewidth=2, markersize=6)

    ax1.set_xlabel('Доля удаленных ребер', fontsize=12)
    ax1.set_ylabel('LCC (размер крупнейшей компоненты)', fontsize=12)
    ax1.set_title(f'Перколяция: LCC ({graph_name})', fontsize=14, weight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(alpha=0.3, linestyle='--')
    ax1.set_xlim(-0.05, 1.05)
    ax1.set_ylim(-0.05, 1.05)

    # Efficiency
    for strategy in ['Targeted', 'Random']:
        data = results[(results['strategy'] == strategy) &
                      (results['metric_type'] == 'Efficiency')]
        ax2.plot(data['fraction_removed'], data['value'],
                marker='s', label=strategy, linewidth=2, markersize=6)

    ax2.set_xlabel('Доля удаленных ребер', fontsize=12)
    ax2.set_ylabel('Глобальная эффективность (E)', fontsize=12)
    ax2.set_title(f'Перколяция: Efficiency ({graph_name})', fontsize=14, weight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(alpha=0.3, linestyle='--')
    ax2.set_xlim(-0.05, 1.05)

    plt.tight_layout()

    plot_file = os.path.join(output_dir, 'percolation_plot.png')
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"   ✓ График сохранен: {plot_file}")
    plt.close()


In [9]:

# ЗАПУСК

# 'zmr' - самый маленький, быстрый
# 'sao' - средний
# 'moscow' - большой

GRAPH_NAME = 'moscow'
DATA_DIR = 'map_data'
OUTPUT_DIR = 'results'

print("\n🎯 Выбран граф:", GRAPH_NAME.upper())
print("📂 Директория с данными:", DATA_DIR)
print("📂 Директория для результатов:", OUTPUT_DIR)
print()

# Запуск полного пайплайна
results = main_pipeline(GRAPH_NAME, DATA_DIR, OUTPUT_DIR)

print("\n" + "="*80)
print("🎉 ВСЕ ГОТОВО!")
print("="*80)
print(f"\n📊 Результаты находятся в: {OUTPUT_DIR}/{GRAPH_NAME}/")
print(f"   - percolation_results.csv  (все данные)")
print(f"   - percolation_plot.png     (графики)")
print(f"   - initial_distances.csv    (исходные пути)")
print(f"   - criticality_scores.csv   (критичность ребер)")
print(f"   - summary.txt              (краткая статистика)")

# Превью результатов
print("\n📈 Превью финальных результатов:")
print(results.head(10).to_string())


🎯 Выбран граф: MOSCOW
📂 Директория с данными: map_data
📂 Директория для результатов: results

🚀 СТАРТ АНАЛИЗА ФУНКЦИОНАЛЬНОЙ УСТОЙЧИВОСТИ: MOSCOW
📥 Загрузка графа из map_data/moscow.pkl...
   ✓ Загружено: 15632 узлов, 32044 рёбер

📊 ШАГ 1: РАСЧЕТ ИСХОДНЫХ РАССТОЯНИЙ
🚀 Загрузка O-D пар из map_data/moscow_od_pairs.csv...
📐 Расчет кратчайших путей для 1000 пар...


Обработка пар: 100%|██████████| 1000/1000 [00:34<00:00, 29.03it/s]



✅ Результаты сохранены в results/moscow/initial_distances.csv
📈 Статистика:
   - Всего пар: 1000
   - Связных пар: 990
   - Несвязных пар: 10
   - Среднее время: 21.51 мин
   - Медиана: 21.35 мин
   - Макс: 38.71 мин

⚡ ШАГ 2: РАСЧЁТ КРИТИЧНОСТИ РЁБЕР (W_e)
📦 Всего ребер для анализа: 32044
📍 Валидных O-D пар: 990/1000
🧠 Предвычисление SPT от 10 источников...


SPT деревья: 100%|██████████| 10/10 [00:01<00:00,  9.56it/s]


✅ SPT построены

🔍 Расчёт критичности 32044 рёбер...



...

   Шаг 10/20 (50%): LCC=0.1425, Eff=0.000228


Целевая атака: 100%|██████████| 20/20 [00:51<00:00,  2.60s/it]


   Шаг 15/20 (75%): LCC=0.0087, Eff=0.000076
   Шаг 20/20 (100%): LCC=0.0002, Eff=0.000000

✅ Целевая атака завершена!

🎲 ШАГ 4: СИМУЛЯЦИЯ СЛУЧАЙНОГО ОТКАЗА
   Количество прогонов: 50
   Шагов на прогон: 20


Прогоны: 100%|██████████| 50/50 [1:12:22<00:00, 86.85s/it]



✅ Случайный отказ завершен! Усреднено 50 прогонов

📊 ФИНАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
✅ Результаты сохранены в results/moscow/percolation_results.csv
📄 Краткая статистика сохранена в results/moscow/summary.txt

📈 Построение графиков...
   ✓ График сохранен: results/moscow/percolation_plot.png

✅ АНАЛИЗ ЗАВЕРШЕН ДЛЯ MOSCOW!

🎉 ВСЕ ГОТОВО!

📊 Результаты находятся в: results/moscow/
   - percolation_results.csv  (все данные)
   - percolation_plot.png     (графики)
   - initial_distances.csv    (исходные пути)
   - criticality_scores.csv   (критичность ребер)
   - summary.txt              (краткая статистика)

📈 Превью финальных результатов:
   fraction_removed  strategy metric_type     value
0              0.00  Targeted         LCC  0.999872
1              0.00  Targeted  Efficiency  0.032187
2              0.05  Targeted         LCC  0.984583
3              0.05  Targeted  Efficiency  0.019657
4              0.10  Targeted         LCC  0.963984
5              0.10  Targeted  Efficiency  0.01315

In [10]:
# data_dir = 'map_data'

# Gs = {
# 	'zmr': load_graph_pickle(os.path.join(data_dir, f'zmr.pkl')),
# 	'sao': load_graph_pickle(os.path.join(data_dir, f'sao.pkl')),
# 	'moscow': load_graph_pickle(os.path.join(data_dir, f'moscow.pkl'))
# }

In [11]:
# crit_scores = pd.read_csv('./results/zmr/criticality_scores.csv')
# for _, score in crit_scores.iloc[:10].iterrows():
# 	edge_data = Gs['zmr'][score['edge_u']][score['edge_v']][score['edge_key']]

# 	if 'geometry' in edge_data:
# 			line = edge_data['geometry']  # shapely LineString
# 			start_coord = line.coords[0]
# 			start_coord = start_coord[1], start_coord[0]   # первая точка
# 			end_coord = line.coords[-1]
# 			end_coord = end_coord[1], end_coord[0]   # последняя точка
# 	# else:
# 	#     # Если geometry нет, берем координаты вершин
# 	#     start_coord = (G.nodes[u]['y'], G.nodes[u]['x'])  # OSMnx использует y=lat, x=lon
# 	#     end_coord = (G.nodes[v]['y'], G.nodes[v]['x'])

# 	print("Start:", start_coord, "End:", end_coord)
